In [0]:
import sys
sys.path.insert(0, 'drive/Graduation Project/Colaboratory')

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 725476945310217342, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 191234048
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 709964908004901152
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [0]:
df = pd.read_csv('drive/Graduation Project/Colaboratory/user_data_anonymized.csv')
df = df.drop([df.columns[0]],axis=1)

In [6]:
df.head()

,user_id,username,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,type,ntrial,timestamp
0,0,User_1,0.032867,-0.011398,-0.989441,0.001340,-0.004726,0.000653,0,3,2018-03-15 21:49:02.853
1,0,User_1,0.036270,-0.646866,-0.749863,0.001340,-0.004726,0.000653,0,3,2018-03-15 21:49:02.876
2,0,User_1,0.036270,-0.646866,-0.749863,0.001340,-0.004726,0.000653,0,3,2018-03-15 21:49:02.903
3,0,User_1,-0.061005,-0.691040,-0.748917,0.320133,0.039149,-0.234852,0,3,2018-03-15 21:49:03.052
4,0,User_1,-0.140427,-0.746765,-0.741333,0.381414,0.046364,-0.343133,0,3,2018-03-15 21:49:03.074


In [0]:
cols = ['sensor_type','trial','name','type','set']
SEQ_LENGTH = 303
OVERLAP = 0
for i in range(SEQ_LENGTH):
    cols.append('x'+str(i))
moving_window_x = pd.DataFrame(columns = cols)

label = pd.DataFrame(columns = ['sensor_type','set','type1','type2','type3','type4'])
s_index = 0
for trial in np.sort(df['ntrial'].unique()):
    nt = df.loc[df['ntrial'] == trial]

    names = df['username'].unique()
    for name in names:
        n = nt.loc[nt['username'] == name]
        
        types = np.array([1,2,3,4])
        for type_ in types:
            t = n.loc[n['type'] == type_]
            
            sensors = t.columns[2:8]
            for sensor in sensors:  
                s = t
                
                if trial == 5: text="test"
                else: text="train"
                    
                row = np.array([sensor])
                row = np.append(row,[trial])
                row = np.append(row,[name])
                row = np.append(row,[type_])
                row = np.append(row,[text])
                if s[sensor].values.shape[0] > SEQ_LENGTH:
                    row = np.append(row,s[sensor].values[:SEQ_LENGTH].astype(float))
                    #print(s[sensor].values[:SEQ_LENGTH])
                elif s[sensor].values.shape[0] <= SEQ_LENGTH:
                    padded_row = np.append(s[sensor].values[:].astype(float),np.array([s[sensor].values[-1].astype(float) for i in range(SEQ_LENGTH-s[sensor].values.shape[0])]))
                    row = np.append(row,padded_row)
                moving_window_x = moving_window_x.append(pd.DataFrame([row],columns=moving_window_x.columns))
                
                row_label = np.array([sensor])
                row_label = np.append(row_label,[text])
                row_label = np.append(row_label,np.zeros((types.shape[0],)))
                row_label[type_+1] = 1.0
                label = label.append(pd.DataFrame([row_label],columns=label.columns))
                
    s_index += s[sensor].shape[0]

In [0]:
moving_window_x['index'] = [i for i in range(moving_window_x.shape[0])]
moving_window_x = moving_window_x.set_index(['index'])

label['index'] = [i for i in range(label.shape[0])]
label = label.set_index(['index'])

In [0]:
train_len = int(moving_window_x.loc[moving_window_x['set'] == 'train'].shape[0]/6)
test_len = int(moving_window_x.loc[moving_window_x['set'] == 'test'].shape[0]/6)

In [0]:
x_train = np.zeros((train_len,SEQ_LENGTH,6))
y_train = np.zeros((train_len,4))

x_test = np.zeros((test_len,SEQ_LENGTH,6))
y_test = np.zeros((test_len,4))

In [0]:
shuffle_data = np.zeros((train_len+test_len,SEQ_LENGTH,6+4))

In [12]:
for i,sensor in enumerate(moving_window_x['sensor_type'].unique()):
    s_train = moving_window_x.loc[moving_window_x['sensor_type'] == sensor]
    s_train = s_train.loc[s_train['set'] == "train"]
    s_test = moving_window_x.loc[moving_window_x['sensor_type'] == sensor]
    s_test = s_test.loc[s_test['set'] == "test"]

    x_train[:,:,i] = s_train.drop(['sensor_type','trial','name','type','set'],axis=1)
    x_test[:,:,i] = s_test.drop(['sensor_type','trial','name','type','set'],axis=1)
    if i == 0:
        l = label.loc[label['sensor_type'] == sensor]
        l_train = l.loc[l['set'] == "train"]
        l_test = l.loc[l['set'] == "test"]
        
        y_train[:,:] = l_train.drop(['sensor_type','set'],axis=1)
        y_test[:,:] = l_test.drop(['sensor_type','set'],axis=1)

print("x_train.shape: "+str(x_train.shape))
print("y_train.shape: "+str(y_train.shape))
print("x_test.shape: "+str(x_test.shape))
print("y_test.shape: "+str(y_test.shape))

x_train.shape: (32, 303, 6)
y_train.shape: (32, 4)
x_test.shape: (8, 303, 6)
y_test.shape: (8, 4)


In [13]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import History
import seaborn as sn
import time

Using TensorFlow backend.


In [14]:
y_train_ = np.zeros((32,303,4))
y_train_[:,0,:] = y_train

y_test_ = np.zeros((8,303,4))
y_test_[:,0,:] = y_test

shuffle_data[:32,:,:6] = x_train
shuffle_data[:32,:,6:] = y_train_

shuffle_data[32:,:,:6] = x_test
shuffle_data[32:,:,6:] = y_test_


while True:
  np.random.shuffle(shuffle_data)

  x_train = shuffle_data[:32,:,:6]
  x_test = shuffle_data[32:,:,:6]
  y_train = shuffle_data[:32,0,6:].reshape(32,4)
  y_test = shuffle_data[32:,0,6:].reshape(8,4)
  if np.sum(y_test[:,0]) == 2:
      if np.sum(y_test[:,1]) == 2:
          if np.sum(y_test[:,2]) == 2:
              if np.sum(y_test[:,3]) == 2:
                  break


batch_size = 1
timesteps = SEQ_LENGTH
data_dim = 6

model = Sequential()
model.add(LSTM(128, return_sequences=True, 
               batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(LSTM(64))
model.add(Dense(32))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


st = time.time()
history = model.fit(x_train, y_train,
          batch_size=batch_size, epochs=100, shuffle=False,
          validation_data=(x_test, y_test),verbose=1)
ft = time.time()

print("batch size : %d -> %d seconds for training with CPU"%(batch_size,ft-st))


Train on 32 samples, validate on 8 samples
Epoch 1/100
32/32 [==============================] - 40s 1s/step - loss: 1.4603 - acc: 0.2188 - val_loss: 1.4013 - val_acc: 0.1250
Epoch 2/100
32/32 [==============================] - 40s 1s/step - loss: 1.3561 - acc: 0.3750 - val_loss: 1.4117 - val_acc: 0.1250
Epoch 3/100
32/32 [==============================] - 39s 1s/step - loss: 1.3135 - acc: 0.4062 - val_loss: 1.3714 - val_acc: 0.1250
Epoch 4/100
32/32 [==============================] - 39s 1s/step - loss: 1.2762 - acc: 0.2812 - val_loss: 1.4087 - val_acc: 0.1250
Epoch 5/100
32/32 [==============================] - 39s 1s/step - loss: 1.2571 - acc: 0.4375 - val_loss: 1.4687 - val_acc: 0.1250
Epoch 6/100
32/32 [==============================] - 39s 1s/step - loss: 1.3042 - acc: 0.4062 - val_loss: 3.2717 - val_acc: 0.1250
Epoch 7/100
 8/32 [======>.......................] - ETA: 26s - loss: 2.8002 - acc: 0.2500

32/32 [==============================] - 39s 1s/step - loss: 1.9299 - acc: 0.1562 - val_loss: 1.7078 - val_acc: 0.1250
Epoch 8/100
32/32 [==============================] - 39s 1s/step - loss: 1.3559 - acc: 0.2812 - val_loss: 1.3545 - val_acc: 0.2500
Epoch 9/100
32/32 [==============================] - 39s 1s/step - loss: 1.2049 - acc: 0.5000 - val_loss: 1.3613 - val_acc: 0.2500
Epoch 10/100
32/32 [==============================] - 38s 1s/step - loss: 1.1339 - acc: 0.4375 - val_loss: 1.3122 - val_acc: 0.2500
Epoch 11/100
32/32 [==============================] - 39s 1s/step - loss: 1.0430 - acc: 0.5000 - val_loss: 1.2202 - val_acc: 0.2500
Epoch 12/100
32/32 [==============================] - 39s 1s/step - loss: 0.9490 - acc: 0.4688 - val_loss: 1.0581 - val_acc: 0.3750
Epoch 13/100
22/32 [===================>..........] - ETA: 11s - loss: 0.7967 - acc: 0.5000

32/32 [==============================] - 39s 1s/step - loss: 0.9772 - acc: 0.5000 - val_loss: 2.1849 - val_acc: 0.1250
Epoch 14/100
32/32 [==============================] - 39s 1s/step - loss: 2.1839 - acc: 0.2500 - val_loss: 1.3932 - val_acc: 0.1250
Epoch 15/100
32/32 [==============================] - 39s 1s/step - loss: 1.3663 - acc: 0.3438 - val_loss: 1.3590 - val_acc: 0.1250
Epoch 16/100
32/32 [==============================] - 39s 1s/step - loss: 1.3346 - acc: 0.3125 - val_loss: 1.3282 - val_acc: 0.1250
Epoch 17/100
32/32 [==============================] - 39s 1s/step - loss: 1.2914 - acc: 0.2500 - val_loss: 1.3688 - val_acc: 0.1250
Epoch 18/100
32/32 [==============================] - 39s 1s/step - loss: 1.2707 - acc: 0.3125 - val_loss: 1.4072 - val_acc: 0.1250
Epoch 19/100
23/32 [====================>.........] - ETA: 10s - loss: 1.2688 - acc: 0.3478

32/32 [==============================] - 39s 1s/step - loss: 1.2403 - acc: 0.3750 - val_loss: 1.4407 - val_acc: 0.1250
Epoch 20/100
32/32 [==============================] - 39s 1s/step - loss: 1.1570 - acc: 0.5625 - val_loss: 1.3581 - val_acc: 0.2500
Epoch 21/100
32/32 [==============================] - 39s 1s/step - loss: 1.5208 - acc: 0.3438 - val_loss: 1.4443 - val_acc: 0.2500
Epoch 22/100
32/32 [==============================] - 39s 1s/step - loss: 1.4592 - acc: 0.2500 - val_loss: 1.4137 - val_acc: 0.2500
Epoch 23/100
32/32 [==============================] - 39s 1s/step - loss: 1.4274 - acc: 0.3438 - val_loss: 1.4004 - val_acc: 0.2500
Epoch 24/100
32/32 [==============================] - 39s 1s/step - loss: 1.3962 - acc: 0.3125 - val_loss: 1.3922 - val_acc: 0.3750
Epoch 25/100
23/32 [====================>.........] - ETA: 10s - loss: 1.4138 - acc: 0.2609

32/32 [==============================] - 39s 1s/step - loss: 1.3624 - acc: 0.3750 - val_loss: 1.2939 - val_acc: 0.3750
Epoch 26/100
32/32 [==============================] - 39s 1s/step - loss: 1.3069 - acc: 0.2188 - val_loss: 1.7660 - val_acc: 0.1250
Epoch 27/100
32/32 [==============================] - 39s 1s/step - loss: 1.2539 - acc: 0.4375 - val_loss: 1.6206 - val_acc: 0.2500
Epoch 28/100
32/32 [==============================] - 39s 1s/step - loss: 1.4781 - acc: 0.3750 - val_loss: 1.4744 - val_acc: 0.2500
Epoch 29/100
32/32 [==============================] - 39s 1s/step - loss: 1.2677 - acc: 0.4062 - val_loss: 1.4618 - val_acc: 0.3750
Epoch 30/100
32/32 [==============================] - 38s 1s/step - loss: 1.1780 - acc: 0.4688 - val_loss: 1.4820 - val_acc: 0.3750
Epoch 31/100
23/32 [====================>.........] - ETA: 10s - loss: 1.1617 - acc: 0.4348

...

32/32 [==============================] - 39s 1s/step - loss: 1.4170 - acc: 0.2812 - val_loss: 1.3524 - val_acc: 0.5000
Epoch 74/100
32/32 [==============================] - 39s 1s/step - loss: 1.3713 - acc: 0.2812 - val_loss: 1.3389 - val_acc: 0.1250
Epoch 75/100
32/32 [==============================] - 39s 1s/step - loss: 1.3472 - acc: 0.2812 - val_loss: 1.3053 - val_acc: 0.3750
Epoch 76/100
32/32 [==============================] - 39s 1s/step - loss: 1.3373 - acc: 0.2812 - val_loss: 1.3403 - val_acc: 0.3750
Epoch 77/100
32/32 [==============================] - 40s 1s/step - loss: 1.4151 - acc: 0.2812 - val_loss: 1.3838 - val_acc: 0.1250
Epoch 78/100
32/32 [==============================] - 39s 1s/step - loss: 1.2182 - acc: 0.4375 - val_loss: 1.0711 - val_acc: 0.6250
Epoch 79/100
23/32 [====================>.........] - ETA: 10s - loss: 1.0969 - acc: 0.4783

32/32 [==============================] - 39s 1s/step - loss: 1.0220 - acc: 0.5312 - val_loss: 0.8939 - val_acc: 0.6250
Epoch 80/100
32/32 [==============================] - 39s 1s/step - loss: 1.4021 - acc: 0.2812 - val_loss: 1.3440 - val_acc: 0.3750
Epoch 81/100
32/32 [==============================] - 39s 1s/step - loss: 1.3494 - acc: 0.2812 - val_loss: 1.2282 - val_acc: 0.3750
Epoch 82/100
32/32 [==============================] - 39s 1s/step - loss: 1.3465 - acc: 0.2188 - val_loss: 1.2455 - val_acc: 0.1250
Epoch 83/100
32/32 [==============================] - 39s 1s/step - loss: 1.3730 - acc: 0.1875 - val_loss: 1.2973 - val_acc: 0.1250
Epoch 84/100
32/32 [==============================] - 39s 1s/step - loss: 1.3299 - acc: 0.2812 - val_loss: 1.2230 - val_acc: 0.3750
Epoch 85/100
23/32 [====================>.........] - ETA: 10s - loss: 1.3519 - acc: 0.2609

32/32 [==============================] - 39s 1s/step - loss: 1.3030 - acc: 0.3438 - val_loss: 1.2275 - val_acc: 0.3750
Epoch 86/100
32/32 [==============================] - 39s 1s/step - loss: 1.3152 - acc: 0.2500 - val_loss: 1.2147 - val_acc: 0.1250
Epoch 87/100
32/32 [==============================] - 39s 1s/step - loss: 1.3094 - acc: 0.2188 - val_loss: 1.2161 - val_acc: 0.1250
Epoch 88/100
32/32 [==============================] - 39s 1s/step - loss: 1.3052 - acc: 0.2188 - val_loss: 1.2193 - val_acc: 0.1250
Epoch 89/100
32/32 [==============================] - 39s 1s/step - loss: 1.3105 - acc: 0.3438 - val_loss: 1.2190 - val_acc: 0.1250
Epoch 90/100
32/32 [==============================] - 39s 1s/step - loss: 1.2730 - acc: 0.1875 - val_loss: 1.2383 - val_acc: 0.3750
Epoch 91/100
23/32 [====================>.........] - ETA: 10s - loss: 1.4777 - acc: 0.2174

32/32 [==============================] - 39s 1s/step - loss: 1.4021 - acc: 0.2812 - val_loss: 1.2682 - val_acc: 0.3750
Epoch 92/100
32/32 [==============================] - 40s 1s/step - loss: 1.3480 - acc: 0.3125 - val_loss: 1.2330 - val_acc: 0.1250
Epoch 93/100
32/32 [==============================] - 40s 1s/step - loss: 1.3133 - acc: 0.3125 - val_loss: 1.2454 - val_acc: 0.1250
Epoch 94/100
32/32 [==============================] - 40s 1s/step - loss: 1.3071 - acc: 0.3438 - val_loss: 1.2605 - val_acc: 0.1250
Epoch 95/100
32/32 [==============================] - 41s 1s/step - loss: 1.2988 - acc: 0.2812 - val_loss: 1.2833 - val_acc: 0.1250
Epoch 96/100
32/32 [==============================] - 40s 1s/step - loss: 1.3091 - acc: 0.2500 - val_loss: 1.2421 - val_acc: 0.1250
Epoch 97/100
23/32 [====================>.........] - ETA: 10s - loss: 1.3483 - acc: 0.2609

32/32 [==============================] - 41s 1s/step - loss: 1.2958 - acc: 0.3750 - val_loss: 1.2577 - val_acc: 0.1250
Epoch 98/100
32/32 [==============================] - 41s 1s/step - loss: 1.2881 - acc: 0.2812 - val_loss: 1.2385 - val_acc: 0.3750
Epoch 99/100
32/32 [==============================] - 40s 1s/step - loss: 1.2694 - acc: 0.3125 - val_loss: 1.2794 - val_acc: 0.3750
Epoch 100/100
32/32 [==============================] - 42s 1s/step - loss: 1.3024 - acc: 0.3438 - val_loss: 1.2401 - val_acc: 0.1250
batch size : 1 -> 3936 seconds for training with CPU


In [15]:
y_train_ = np.zeros((32,303,4))
y_train_[:,0,:] = y_train

y_test_ = np.zeros((8,303,4))
y_test_[:,0,:] = y_test

shuffle_data[:32,:,:6] = x_train
shuffle_data[:32,:,6:] = y_train_

shuffle_data[32:,:,:6] = x_test
shuffle_data[32:,:,6:] = y_test_


while True:
  np.random.shuffle(shuffle_data)

  x_train = shuffle_data[:32,:,:6]
  x_test = shuffle_data[32:,:,:6]
  y_train = shuffle_data[:32,0,6:].reshape(32,4)
  y_test = shuffle_data[32:,0,6:].reshape(8,4)
  if np.sum(y_test[:,0]) == 2:
      if np.sum(y_test[:,1]) == 2:
          if np.sum(y_test[:,2]) == 2:
              if np.sum(y_test[:,3]) == 2:
                  break


batch_size = 8
timesteps = SEQ_LENGTH
data_dim = 6

model = Sequential()
model.add(LSTM(128, return_sequences=True, 
               batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(LSTM(64))
model.add(Dense(32))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


st = time.time()
history = model.fit(x_train, y_train,
          batch_size=batch_size, epochs=100, shuffle=False,
          validation_data=(x_test, y_test),verbose=1)
ft = time.time()

print("batch size : %d -> %d seconds for training with CPU"%(batch_size,ft-st))


Train on 32 samples, validate on 8 samples
Epoch 1/100
32/32 [==============================] - 6s 182ms/step - loss: 1.4285 - acc: 0.2188 - val_loss: 1.4022 - val_acc: 0.2500
Epoch 2/100
32/32 [==============================] - 5s 159ms/step - loss: 1.3751 - acc: 0.2812 - val_loss: 1.3914 - val_acc: 0.1250
Epoch 3/100
32/32 [==============================] - 5s 158ms/step - loss: 1.3480 - acc: 0.3750 - val_loss: 1.3814 - val_acc: 0.1250
Epoch 4/100
32/32 [==============================] - 5s 160ms/step - loss: 1.3180 - acc: 0.4375 - val_loss: 1.3730 - val_acc: 0.1250
Epoch 5/100
32/32 [==============================] - 5s 157ms/step - loss: 1.2732 - acc: 0.5312 - val_loss: 1.3667 - val_acc: 0.2500
Epoch 6/100
32/32 [==============================] - 5s 164ms/step - loss: 1.2006 - acc: 0.4688 - val_loss: 1.2037 - val_acc: 0.3750
Epoch 7/100
32/32 [==============================] - 5s 161ms/step - loss: 1.0846 - acc: 0.4062 - val_loss: 0.9871 - val_acc: 0.3750
Epoch 8/100
32/32 [=======

32/32 [==============================] - 5s 158ms/step - loss: 1.1924 - acc: 0.5625 - val_loss: 1.4062 - val_acc: 0.7500
Epoch 18/100
32/32 [==============================] - 5s 160ms/step - loss: 0.8736 - acc: 0.5000 - val_loss: 1.5381 - val_acc: 0.3750
Epoch 19/100
32/32 [==============================] - 5s 158ms/step - loss: 0.6619 - acc: 0.6562 - val_loss: 1.4384 - val_acc: 0.3750
Epoch 20/100
32/32 [==============================] - 5s 159ms/step - loss: 0.6270 - acc: 0.7188 - val_loss: 1.4711 - val_acc: 0.3750
Epoch 21/100
32/32 [==============================] - 5s 159ms/step - loss: 0.5241 - acc: 0.7812 - val_loss: 1.5210 - val_acc: 0.6250
Epoch 22/100
32/32 [==============================] - 5s 160ms/step - loss: 0.5005 - acc: 0.8438 - val_loss: 1.5560 - val_acc: 0.6250
Epoch 23/100
32/32 [==============================] - 5s 153ms/step - loss: 0.4778 - acc: 0.9062 - val_loss: 1.5878 - val_acc: 0.6250
Epoch 24/100
32/32 [==============================] - 5s 155ms/step - loss:

32/32 [==============================] - 5s 159ms/step - loss: 0.4328 - acc: 0.8125 - val_loss: 1.0999 - val_acc: 0.7500
Epoch 34/100
32/32 [==============================] - 5s 159ms/step - loss: 0.3841 - acc: 0.8438 - val_loss: 1.1031 - val_acc: 0.7500
Epoch 35/100
32/32 [==============================] - 5s 163ms/step - loss: 0.3743 - acc: 0.8750 - val_loss: 1.1368 - val_acc: 0.6250
Epoch 36/100
32/32 [==============================] - 5s 167ms/step - loss: 0.2928 - acc: 0.9375 - val_loss: 1.1624 - val_acc: 0.6250
Epoch 37/100
32/32 [==============================] - 5s 166ms/step - loss: 0.2814 - acc: 0.9375 - val_loss: 1.2930 - val_acc: 0.6250
Epoch 38/100
32/32 [==============================] - 5s 167ms/step - loss: 0.2812 - acc: 0.8750 - val_loss: 1.3199 - val_acc: 0.6250
Epoch 39/100
32/32 [==============================] - 5s 162ms/step - loss: 0.2308 - acc: 0.9375 - val_loss: 1.3224 - val_acc: 0.6250
Epoch 40/100
32/32 [==============================] - 5s 165ms/step - loss:

32/32 [==============================] - 5s 165ms/step - loss: 0.6712 - acc: 0.8125 - val_loss: 1.8179 - val_acc: 0.6250
Epoch 51/100
32/32 [==============================] - 5s 165ms/step - loss: 0.4210 - acc: 0.8438 - val_loss: 1.7639 - val_acc: 0.5000
Epoch 52/100
32/32 [==============================] - 5s 162ms/step - loss: 0.2501 - acc: 0.9062 - val_loss: 1.4120 - val_acc: 0.7500
Epoch 53/100
32/32 [==============================] - 5s 162ms/step - loss: 0.1604 - acc: 0.9375 - val_loss: 1.4270 - val_acc: 0.6250
Epoch 54/100
32/32 [==============================] - 5s 164ms/step - loss: 0.1486 - acc: 0.9688 - val_loss: 1.3938 - val_acc: 0.6250
Epoch 55/100
32/32 [==============================] - 5s 164ms/step - loss: 0.1176 - acc: 0.9688 - val_loss: 1.5664 - val_acc: 0.6250
Epoch 56/100
32/32 [==============================] - 5s 162ms/step - loss: 0.0999 - acc: 0.9688 - val_loss: 1.5717 - val_acc: 0.5000
Epoch 57/100
32/32 [==============================] - 5s 163ms/step - loss:

32/32 [==============================] - 5s 164ms/step - loss: 0.0182 - acc: 1.0000 - val_loss: 2.2035 - val_acc: 0.5000
Epoch 68/100
32/32 [==============================] - 5s 163ms/step - loss: 0.0165 - acc: 1.0000 - val_loss: 2.2356 - val_acc: 0.5000
Epoch 69/100
32/32 [==============================] - 5s 162ms/step - loss: 0.0151 - acc: 1.0000 - val_loss: 2.2610 - val_acc: 0.5000
Epoch 70/100
32/32 [==============================] - 5s 167ms/step - loss: 0.0139 - acc: 1.0000 - val_loss: 2.2804 - val_acc: 0.5000
Epoch 71/100
32/32 [==============================] - 5s 163ms/step - loss: 0.0128 - acc: 1.0000 - val_loss: 2.2991 - val_acc: 0.5000
Epoch 72/100
32/32 [==============================] - 5s 164ms/step - loss: 0.0118 - acc: 1.0000 - val_loss: 2.3201 - val_acc: 0.5000
Epoch 73/100
32/32 [==============================] - 5s 163ms/step - loss: 0.0110 - acc: 1.0000 - val_loss: 2.3426 - val_acc: 0.5000
Epoch 74/100
32/32 [==============================] - 5s 164ms/step - loss:

32/32 [==============================] - 5s 165ms/step - loss: 0.0059 - acc: 1.0000 - val_loss: 2.5670 - val_acc: 0.5000
Epoch 85/100
32/32 [==============================] - 5s 164ms/step - loss: 0.0056 - acc: 1.0000 - val_loss: 2.5857 - val_acc: 0.5000
Epoch 86/100
32/32 [==============================] - 5s 164ms/step - loss: 0.0054 - acc: 1.0000 - val_loss: 2.6042 - val_acc: 0.5000
Epoch 87/100
32/32 [==============================] - 5s 162ms/step - loss: 0.0052 - acc: 1.0000 - val_loss: 2.6225 - val_acc: 0.5000
Epoch 88/100
32/32 [==============================] - 5s 165ms/step - loss: 0.0050 - acc: 1.0000 - val_loss: 2.6404 - val_acc: 0.5000
Epoch 89/100
32/32 [==============================] - 5s 164ms/step - loss: 0.0048 - acc: 1.0000 - val_loss: 2.6579 - val_acc: 0.5000
Epoch 90/100
32/32 [==============================] - 5s 163ms/step - loss: 0.0046 - acc: 1.0000 - val_loss: 2.6751 - val_acc: 0.5000
Epoch 91/100
32/32 [==============================] - 5s 163ms/step - loss: